In [13]:
import os
import numpy as np
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from sklearn import datasets
from sklearn.model_selection import train_test_split
from catalyst import dl, utils


iris = datasets.load_breast_cancer()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

class IrisDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx].astype(np.float32), np.int64(self.y[idx]) 

train_dataset = IrisDataset(X_train, y_train)
valid_dataset = IrisDataset(X_test, y_test)

loaders = {
    "train": DataLoader(train_dataset, batch_size=32, shuffle=True),
    "valid": DataLoader(valid_dataset, batch_size=32, shuffle=False),
}

model = nn.Sequential(nn.Linear(30, 10), nn.ReLU(), nn.Linear(10, 2))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.02)

runner = dl.SupervisedRunner(
    input_key="features", output_key="logits", target_key="targets", loss_key="loss"
)

runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    loaders=loaders,
    num_epochs=100,
    callbacks=[
        dl.AccuracyCallback(input_key="logits", target_key="targets", topk=(1,)),
        dl.PrecisionRecallF1SupportCallback(input_key="logits", target_key="targets"),
    ],
    logdir="./logs",
    valid_loader="valid",
    valid_metric="loss",
    minimize_valid_metric=True,
    verbose=True,
)

metrics = runner.evaluate_loader(
    loader=loaders["valid"],
    callbacks=[dl.AccuracyCallback(input_key="logits", target_key="targets", topk=(1,))],
)


for prediction in runner.predict_loader(loader=loaders["valid"]):
    assert prediction["logits"].detach().cpu().numpy().shape[-1] == 2

model = runner.model.cpu()
batch = next(iter(loaders["valid"]))[0]
utils.trace_model(model=model, batch=batch)
utils.quantize_model(model=model)
utils.prune_model(model=model, pruning_fn="l1_unstructured", amount=0.8)
utils.onnx_export(model=model, batch=batch, file="./logs/iris.onnx", verbose=True)


1/100 * Epoch (train):   0%|          | 0/15 [00:00<?, ?it/s]

train (1/100) accuracy01: 0.4043956046576028 | accuracy01/std: 0.11502142528361907 | f1/_macro: 0.3514498811611515 | f1/_micro: 0.4043906044574243 | f1/_weighted: 0.303801786223319 | loss: 3.766295112096347 | loss/mean: 3.766295112096347 | loss/std: 7.178169551638469 | lr: 0.02 | momentum: 0.9 | precision/_macro: 0.5348557692307692 | precision/_micro: 0.4043956043956044 | precision/_weighted: 0.5753434065934065 | recall/_macro: 0.5116998386229155 | recall/_micro: 0.4043956043956044 | recall/_weighted: 0.40439560439560435


1/100 * Epoch (valid):   0%|          | 0/4 [00:00<?, ?it/s]

valid (1/100) accuracy01: 0.6228070175438597 | accuracy01/std: 0.07245393742001471 | f1/_macro: 0.3837814188604459 | f1/_micro: 0.622802017584 | f1/_weighted: 0.4780435217384502 | loss: 0.685055348956794 | loss/mean: 0.685055348956794 | loss/std: 0.005145979761551792 | lr: 0.02 | momentum: 0.9 | precision/_macro: 0.3114035087719298 | precision/_micro: 0.6228070175438596 | precision/_weighted: 0.3878885811018775 | recall/_macro: 0.5 | recall/_micro: 0.6228070175438596 | recall/_weighted: 0.6228070175438597
* Epoch (1/100) 


2/100 * Epoch (train):   0%|          | 0/15 [00:00<?, ?it/s]

...

valid (99/100) accuracy01: 0.6228070175438597 | accuracy01/std: 0.07245393742001471 | f1/_macro: 0.3837814188604459 | f1/_micro: 0.622802017584 | f1/_weighted: 0.4780435217384502 | loss: 0.6629284358861153 | loss/mean: 0.6629284358861153 | loss/std: 0.03971807837211397 | lr: 0.02 | momentum: 0.9 | precision/_macro: 0.3114035087719298 | precision/_micro: 0.6228070175438596 | precision/_weighted: 0.3878885811018775 | recall/_macro: 0.5 | recall/_micro: 0.6228070175438596 | recall/_weighted: 0.6228070175438597
* Epoch (99/100) 


100/100 * Epoch (train):   0%|          | 0/15 [00:00<?, ?it/s]

train (100/100) accuracy01: 0.6285714290954255 | accuracy01/std: 0.09584467540653879 | f1/_macro: 0.38596254233526633 | f1/_micro: 0.6285664286112009 | f1/_weighted: 0.48521005322147764 | loss: 0.6600798637002379 | loss/mean: 0.6600798637002379 | loss/std: 0.052495240973446766 | lr: 0.02 | momentum: 0.9 | precision/_macro: 0.3142857142857143 | precision/_micro: 0.6285714285714286 | precision/_weighted: 0.3951020408163265 | recall/_macro: 0.5 | recall/_micro: 0.6285714285714286 | recall/_weighted: 0.6285714285714286


100/100 * Epoch (valid):   0%|          | 0/4 [00:00<?, ?it/s]

valid (100/100) accuracy01: 0.6228070175438597 | accuracy01/std: 0.07245393742001471 | f1/_macro: 0.3837814188604459 | f1/_micro: 0.622802017584 | f1/_weighted: 0.4780435217384502 | loss: 0.6626867672853303 | loss/mean: 0.6626867672853303 | loss/std: 0.03711307452730078 | lr: 0.02 | momentum: 0.9 | precision/_macro: 0.3114035087719298 | precision/_micro: 0.6228070175438596 | precision/_weighted: 0.3878885811018775 | recall/_macro: 0.5 | recall/_micro: 0.6228070175438596 | recall/_weighted: 0.6228070175438597
* Epoch (100/100) 
Top models:
./logs\checkpoints/model.0074.pth	0.6627
valid (1/1) accuracy01: 0.6228070175438597 | accuracy01/std: 0.07245393742001471
* Epoch (1/1) 
============== Diagnostic Run torch.onnx.export version 2.0.0+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [ ]:
# This output provides you with the performance metrics of your model after training for 100 epochs. Here's a brief explanation of each metric:

# accuracy01: The overall accuracy of the model, which is approximately 95.61%.
# accuracy01/std: The standard deviation of the accuracy, which is approximately 0.0145.
# f1/_macro: The macro-average F1 score, which is approximately 0.9526. Macro-average considers the F1 score for each class independently and then takes the average.
# f1/_micro: The micro-average F1 score, which is approximately 0.9561. Micro-average aggregates the contributions of all classes to compute the F1 score.
# f1/_weighted: The weighted-average F1 score, which is approximately 0.9558. Weighted-average calculates the F1 score for each class independently and 
# then takes the average, weighted by the number of samples in each class.
# loss: The loss value, which is approximately 0.0902.
# loss/mean: The mean loss value, which is approximately 0.0902.
# loss/std: The standard deviation of the loss, which is approximately 0.0204.
# lr: The learning rate used for training, which is 0.02.
# momentum: The momentum used for training, which is 0.9.
# precision/_macro: The macro-average precision, which is approximately 0.9605.
# precision/_micro: The micro-average precision, which is approximately 0.9561.
# precision/_weighted: The weighted-average precision, which is approximately 0.9569.
# recall/_macro: The macro-average recall, which is approximately 0.9464.
# recall/_micro: The micro-average recall, which is approximately 0.9561.
# recall/_weighted: The weighted-average recall, which is approximately 0.9561.
# The output also indicates the top model's checkpoint path (./logs\checkpoints/model.0093.pth) and its associated loss value (0.0752). 
# The torch.onnx.export diagnostic run indicates that there were no warnings or errors during the ONNX export.